# A spatial Stochastic SEIR model using the framework

## Rules:

### Intralocation rules
Exposure: S -> E

Infection: E -> I

Recovery: I -> R
### Translocation rules
Movement (Susceptible, Exposed, Recovered): S_{L1} -> S_{L2}, E_{L1} -> E_{L2}, R_{L1} -> R_{L2}

Movement (Infected): I_{L1} -> I_{L2}

In [37]:
import numpy as np

import BasicRules, ModelDefinition, ModelLocations


In [38]:
epiClasses = [["S", "people"], ["E", "people"], ["I", "people"], ["R", "people"]]

In [39]:
model_constants = {
    "Exposure_rate":2,
    "Infection_rate":1,
    "Recovery_rate":0.2,
    "Death_proportion":0.01,
    "Standard_Transport_prop":0.001,
    "Infected_Transport_prop":0.0001
}

In [40]:
def epidemologyRules():
    # Frequency vs prevalance - /N but can also use a frequency propensity (fits some dieases better than others).
    exposure = BasicRules.SingleLocationProductionRule("Region", ["S"], [1], ["E"], [1], 
                                                              f"loc_regional_inf*{model_constants['Exposure_rate']}*I*S/(S+E+I+R)*model_month_jan +loc_regional_inf*{model_constants['Exposure_rate']}*I*S/(S+E+I+R)*model_month_feb",
                                                              ["S","E","I","R"], "Exposure")
    infection = BasicRules.SingleLocationProductionRule("Region", ["E"], [1], ["I"], [1], 
                                                              f"{model_constants['Infection_rate']}*E",
                                                              ["E"], "Infection")    
    recovery = BasicRules.SingleLocationProductionRule("Region", ["I"], [1], ["R"], [1],
                                                            f"{model_constants['Recovery_rate']}*I",
                                                            ["I"], "Recovery")
    death = BasicRules.ExitEntranceRule("Region", "I", 1, 
                                              f"{model_constants['Death_proportion']}*{model_constants['Recovery_rate']}*I", ["I"], "Death")
    
    transports = []
    for transport in ["S","E","I","R"]:
        if transport == "I":
            relocation_rate = model_constants["Infected_Transport_prop"]
        else:
            relocation_rate = model_constants["Standard_Transport_prop"]

        transports.append(BasicRules.TransportRule("Region", "Region", transport, [f"{str(relocation_rate)}*{transport}","1"], 
                                                     1, [[transport], ["S"]], "Movement"))
        
    return [exposure, infection, recovery, death] #+ transports

In [41]:
class Region (ModelLocations.Location):
    def __init__(self, lat, long, name):
        # Sets lat/long and creates and empty set of compartment labels.
        super().__init__(lat, long, name, loc_type="Region", constants=["regional_inf"])
        self.class_labels.add("S")
        self.class_labels.add("E")
        self.class_labels.add("I")
        self.class_labels.add("R")

def sirLocations():
    all_locations = []
    # Midpoints from wikipedia, South East and London uses South East midpoint (combined to match DEFRA reporting)
    region_infos = [[0, 0, "Test Region"], [54.075, -2.75, "North East"], [55, -1.87, "North West"], [53.566667, -1.2, "Yorkshire & The Humber"], [52.98, -0.75, "East Midlands"], [52.478861, -2.256306, "West Midlands"], 
                    [52.24, 0.41, "East of England"], [51.515447, -0.09214, "London"], [51.3, -0.8, "South East"], [50.96, -3.22, "South West"], [56.816738, -4.183963, "Scotland"], 
                    [52.33022, -3.766409,"Wales"]]
    # ONS mid 2022 estimates
    region_populations = list(0.001*np.array([10000, 2683040, 7516113, 5541262, 4934939, 6021653, 6398497, 8866180, 9379833, 5764881, 5447700, 3131640]))
    for region_index, region_info in enumerate(region_infos):
        region = Region(*region_info)
        # Defaults to zero otherwise
        region.setInitialConditions({"S":region_populations[region_index],"I":100})
        if region_index == 0:
            region.setConstants({"regional_inf":0})
        else:
            region.setConstants({"regional_inf":0.2})
        all_locations.append(region)

    return all_locations

In [42]:
model = ModelDefinition.ModelDefinition(epiClasses, sirLocations, epidemologyRules, model_folder="../ModelFiles/RegionalEpi/")
model.build()

loc_regional_inf*2*I*S/(S+E+I+R)*model_month_jan +loc_regional_inf*2*I*S/(S+E+I+R)*model_month_feb
1*E
0.2*I
0.01*0.2*I
0.001*S
1
0.001*E
1
0.0001*I
1
0.001*R
1
1.00000000000000
['S', 'E']
1.00000000000000
['E', 'I']
0.200000000000000
['I', 'R']
0.00200000000000000
['I']
MATCHED {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}
MATCHED {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}
MATCHED {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}
MATCHED {0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}
['loc_regional_inf*2*x1*x3/(x3+x0+x1+x2)*x10 +loc_regional_inf*2*x1*x3/(x3+x0+x1+x2)*x9']
['1*x0']
['0.2*x1']
['0.01*0.2*x1']


: 